In [5]:
import json

response = {
  "id": "msg_01SQxKnt4JPv29XtmxdFrby8",
  "type": "message",
  "role": "assistant",
  "model": "claude-opus-4-5-20251101",
  "content": [
    {
      "type": "text",
      "text": "# 📚 Data Drift Detection: Complete Guide for MLOps\n\n## สารบัญ\n1. [บทนำ: ทำความเข้าใจ Data Drift](#บทนำ-ทำความเข้าใจ-data-drift)\n2. [LAB 1: Understanding Data Drift Concepts](#lab-1-understanding-data-drift-concepts)\n3. [LAB 2: Feature Drift Detection](#lab-2-feature-drift-detection)\n4. [LAB 3: Multivariate Drift Analysis](#lab-3-multivariate-drift-analysis)\n5. [LAB 4: Drift Detection in Production Simulation](#lab-4-drift-detection-in-production-simulation)\n6. [LAB 5: Custom Metrics & Drift Thresholds](#lab-5-custom-metrics--drift-thresholds)\n7. [LAB 6: End-to-End Monitoring Pipeline](#lab-6-end-to-end-monitoring-pipeline)\n8. [สรุปและ Best Practices](#สรุปและ-best-practices)\n\n---\n\n## บทนำ: ทำความเข้าใจ Data Drift\n\n### Data Drift คืออะไร?\n\n**Data Drift** (หรือ Dataset Shift) คือปรากฏการณ์ที่ข้อมูลในระบบ production มีการเปลี่ยนแปลงไปจากข้อมูลที่ใช้ train โมเดล ซึ่งเป็นหนึ่งในสาเหตุหลักที่ทำให้ ML Model มีประสิทธิภาพลดลงเมื่อเวลาผ่านไป\n\n```\nTraining Time                    Production Time\n     │                                │\n     ▼                                ▼\n┌─────────────┐               ┌─────────────┐\n│ Training    │               │ Production  │\n│ Data        │──── Drift ───▶│ Data        │\n│ P(X,Y)_train│               │ P(X,Y)_prod │\n└─────────────┘               └─────────────┘\n     │                                │\n     ▼                                ▼\n┌─────────────┐               ┌─────────────┐\n│ Good Model  │               │ Degraded    │\n│ Performance │               │ Performance │\n└─────────────┘               └─────────────┘\n```\n\n### ทำไม Drift Detection ถึงสำคัญ?\n\n1. **Model Degradation**: โมเดลที่ train ด้วยข้อมูลเก่าอาจทำนายผิดพลาดกับข้อมูลใหม่\n2. **Business Impact**: การตัดสินใจผิดพลาดส่งผลต่อธุรกิจโดยตรง\n3. **Regulatory Compliance**: หลายอุตสาหกรรมต้องการ monitoring อย่างต่อเนื่อง\n4. **Resource Optimization**: รู้เวลาที่ต้อง retrain ช่วยประหยัดทรัพยากร\n\n---\n\n## LAB 1: Understanding Data Drift Concepts\n\n### 🎯 วัตถุประสงค์\n- เข้าใจความแตกต่างระหว่าง Covariate Shift และ Concept Drift\n- เรียนรู้ Statistical tests สำหรับ drift detection\n- สามารถเลือก drift detection method ที่เหมาะสม\n\n### ทฤษฎี: ประเภทของ Data Drift\n\n#### 1. Covariate Shift (Feature Drift)\n\n**นิยาม**: การเปลี่ยนแปลงของ distribution ของ input features P(X) โดยที่ความสัมพันธ์ P(Y|X) ยังคงเดิม\n\n```\nCovariate Shift:\n├── P(X)_train ≠ P(X)_prod     ← Distribution เปลี่ยน\n└── P(Y|X)_train = P(Y|X)_prod  ← Relationship คงเดิม\n```\n\n**ตัวอย่างในชีวิตจริง**:\n- โมเดลทำนายราคาบ้านที่ train กับบ้านในเมือง แต่ต้องทำนายบ้านในชนบท\n- โมเดลอายุลูกค้าที่ train กับกลุ่มอายุ 20-40 ปี แต่ production มีลูกค้าอายุ 40-60 ปี\n\n**Code Reference** - การสร้างข้อมูล Covariate Shift:\n```python\ndef generate_covariate_shift_data():\n    # Training data: ลูกค้าอายุน้อย (20-40)\n    age_train = np.random.normal(30, 5, n_train)\n    \n    # Production data: ลูกค้าอายุมากขึ้น (40-60) - Covariate Shift!\n    age_prod = np.random.normal(50, 5, n_prod)\n    \n    # กฎการซื้อเหมือนเดิม (ไม่มี Concept Drift)\n    # P(Y|X) ยังคงเดิม\n```\n\n#### 2. Concept Drift (Label Drift)\n\n**นิยาม**: การเปลี่ยนแปลงของความสัมพันธ์ระหว่าง input และ output P(Y|X)\n\n```\nConcept Drift:\n├── P(X) อาจคงที่หรือเปลี่ยนก็ได้\n└── P(Y|X)_train ≠ P(Y|X)_prod  ← Relationship เปลี่ยน!\n```\n\n**ตัวอย่างในชีวิตจริง**:\n- พฤติกรรมการซื้อของลูกค้าเปลี่ยนหลัง COVID-19\n- ความหมายของ \"spam email\" เปลี่ยนไปตามเวลา\n\n**Code Reference** - การสร้างข้อมูล Concept Drift:\n```python\ndef generate_concept_drift_data():\n    # Training: กฎการซื้อเดิม - ซื้อถ้า income > 45000\n    purchase_train = (income_train > 45000).astype(int)\n    \n    # Production: กฎการซื้อเปลี่ยน - ซื้อถ้า income > 55000\n    # Threshold เปลี่ยน = Concept Drift!\n    purchase_prod = (income_prod > 55000).astype(int)\n```\n\n#### เปรียบเทียบ Covariate Shift vs Concept Drift\n\n| ลักษณะ | Covariate Shift | Concept Drift |\n|--------|-----------------|---------------|\n| สิ่งที่เปลี่ยน | P(X) | P(Y\\|X) |\n| ตัวอย่าง | กลุ่มลูกค้าเปลี่ยน | พฤติกรรมเปลี่ยน |\n| การตรวจจับ | เปรียบเทียบ feature distributions | ต้องมี labels หรือ performance |\n| วิธีแก้ | Sample weighting, Retrain | Retrain with new data |\n\n---\n\n### Statistical Tests สำหรับ Drift Detection\n\n#### 1. Kolmogorov-Smirnov (KS) Test\n\n**ทฤษฎี**:\n- เปรียบเทียบ Cumulative Distribution Function (CDF) ของ 2 samples\n- วัดความแตกต่างสูงสุดระหว่าง 2 CDFs\n\n```\nKS Statistic = max|F₁(x) - F₂(x)|\n\nโดยที่:\n- F₁(x) = CDF ของ reference data\n- F₂(x) = CDF ของ current data\n```\n\n**การตีความ**:\n- KS Statistic: 0-1 (ยิ่งสูง = ยิ่งแตกต่าง)\n- p-value < 0.05: reject null hypothesis → มี drift\n\n**ข้อดี**:\n- ไม่ต้องสมมติ distribution (non-parametric)\n- Sensitive ต่อการเปลี่ยนแปลง\n\n**ข้อเสีย**:\n- ใช้ได้กับ continuous variables เท่านั้น\n\n**Code Reference**:\n```python\ndef kolmogorov_smirnov_test(data1, data2, feature_name=\"feature\"):\n    statistic, p_value = stats.ks_2samp(data1, data2)\n    drift_detected = p_value < 0.05\n    \n    return {\n        'statistic': statistic,\n        'p_value': p_value,\n        'drift_detected': drift_detected\n    }\n```\n\n#### 2. Population Stability Index (PSI)\n\n**ทฤษฎี**:\n- วัดการเปลี่ยนแปลงของ distribution โดยเปรียบเทียบ proportions ในแต่ละ bin\n- นิยมใช้ใน credit scoring และ financial models\n\n**สูตร**:\n```\nPSI = Σ (Actual% - Expected%) × ln(Actual% / Expected%)\n\nโดยที่:\n- Expected% = proportion ใน reference data\n- Actual% = proportion ใน current data\n```\n\n**การตีความ PSI**:\n\n| ค่า PSI | ความหมาย | Action |\n|---------|----------|--------|\n| < 0.1 | ไม่มีการเปลี่ยนแปลงสำคัญ | ปกติ |\n| 0.1 - 0.25 | การเปลี่ยนแปลงปานกลาง | ตรวจสอบเพิ่ม |\n| ≥ 0.25 | การเปลี่ยนแปลงมาก | ต้องดำเนินการ |\n\n**Code Reference**:\n```python\ndef calculate_psi(expected, actual, bins=10, eps=1e-6):\n    # สร้าง bins จาก expected data\n    breakpoints = np.percentile(expected, np.linspace(0, 100, bins + 1))\n    \n    # นับจำนวนในแต่ละ bin\n    expected_counts, _ = np.histogram(expected, bins=breakpoints)\n    actual_counts, _ = np.histogram(actual, bins=breakpoints)\n    \n    # คำนวณ proportions\n    expected_props = expected_counts / len(expected) + eps\n    actual_props = actual_counts / len(actual) + eps\n    \n    # คำนวณ PSI\n    psi = np.sum((actual_props - expected_props) * np.log(actual_props / expected_props))\n    return psi\n```\n\n#### 3. Wasserstein Distance (Earth Mover's Distance)\n\n**ทฤษฎี**:\n- วัด \"งาน\" ที่ต้องใช้ในการเปลี่ยน distribution หนึ่งไปเป็นอีก distribution\n- คล้ายกับการคำนวณต้นทุนในการขนย้ายดิน\n\n```\nWasserstein Distance = inf ∫|F₁⁻¹(u) - F₂⁻¹(u)| du\n\nเปรียบเสมือน:\n┌─────────────┐         ┌─────────────┐\n│   Pile A    │  move   │   Pile B    │\n│   (sand)    │ ──────▶ │   (sand)    │\n└─────────────┘  cost   └─────────────┘\n```\n\n**ข้อดี**:\n- คำนึงถึง distance ระหว่าง bins\n- Sensitive ต่อ shift ในตำแหน่ง\n\n**Code Reference**:\n```python\ndef wasserstein_distance_test(data1, data2, feature_name=\"feature\"):\n    distance = stats.wasserstein_distance(data1, data2)\n    \n    # Normalize โดยใช้ standard deviation\n    std_ref = np.std(data1)\n    normalized_distance = distance / std_ref if std_ref > 0 else distance\n    \n    return {\n        'distance': distance,\n        'normalized_distance': normalized_distance\n    }\n```\n\n### การเลือก Drift Detection Method\n\n```\nDecision Tree:\n\n1. Data Type?\n   ├── Continuous → ไปข้อ 2\n   └── Categorical → Chi-squared หรือ PSI\n\n2. ต้องการ Statistical Significance?\n   ├── ใช่ → KS Test\n   └── ไม่จำเป็น → PSI หรือ Wasserstein\n\n3. Industry Requirement?\n   ├── Finance/Credit → PSI (regulatory standards)\n   └── อื่นๆ → เลือกตามความเหมาะสม\n\n4. ต้องการ Sensitivity สูง?\n   ├── ใช่ → Wasserstein\n   └── ปกติ → KS หรือ PSI\n```\n\n---\n\n## LAB 2: Feature Drift Detection\n\n### 🎯 วัตถุประสงค์\n- ตรวจจับ drift ในแต่ละ feature อย่างเป็นระบบ\n- วิเคราะห์ numerical vs categorical feature drift\n- สร้าง visualization สำหรับ feature distributions\n\n### ทฤษฎี: Per-Feature Analysis\n\nการวิเคราะห์ drift ในแต่ละ feature มีความสำคัญเพราะ:\n1. ช่วยระบุ root cause ของ model performance degradation\n2. ทำให้เข้าใจว่า feature ไหนเปลี่ยนแปลงมากที่สุด\n3. สามารถ prioritize การแก้ไขได้\n\n### Numerical Features vs Categorical Features\n\n#### Numerical Features\n```\nMethods ที่ใช้:\n├── KS Test - compare CDFs\n├── PSI - compare bin proportions\n└── Wasserstein - measure distribution distance\n```\n\n#### Categorical Features\n```\nMethods ที่ใช้:\n├── Chi-squared Test - compare frequency distributions\n└── PSI (category-based) - compare category proportions\n```\n\n### Feature Drift Detector Class\n\n**Architecture**:\n```\nFeatureDriftDetector\n├── __init__()\n│   ├── reference_data\n│   ├── current_data\n│   ├── numerical_features\n│   └── categorical_features\n│\n├── Numerical Methods\n│   ├── ks_test()\n│   ├── calculate_psi()\n│   └── wasserstein_test()\n│\n├── Categorical Methods\n│   ├── chi_squared_test()\n│   └── categorical_psi()\n│\n└── Analysis\n    ├── analyze_numerical_feature()\n    ├── analyze_categorical_feature()\n    ├── analyze_all_features()\n    └── get_summary_report()\n```\n\n**Code Reference**:\n```python\nclass FeatureDriftDetector:\n    def __init__(self, reference_data, current_data, \n                 numerical_features=None, categorical_features=None):\n        self.reference = reference_data\n        self.current = current_data\n        self.numerical_features = numerical_features\n        self.categorical_features = categorical_features\n    \n    def analyze_numerical_feature(self, feature):\n        results = {\n            'ks_test': self.ks_test(feature),\n            'psi': self.calculate_psi(feature),\n            'wasserstein': self.wasserstein_test(feature)\n        }\n        # สรุปผล\n        ks_drift = results['ks_test']['p_value'] < 0.05\n        psi_value = results['psi']['psi']\n        \n        if psi_value < 0.1:\n            psi_severity = 'none'\n        elif psi_value < 0.25:\n            psi_severity = 'mild'\n        else:\n            psi_severity = 'severe'\n        \n        results['drift_detected'] = ks_drift or psi_severity != 'none'\n        return results\n```\n\n### Feature Drift Ranking\n\nจัดลำดับ features ตามความรุนแรงของ drift เพื่อ prioritization:\n\n```\nRanking Algorithm:\n1. คำนวณ PSI สำหรับทุก features\n2. เรียงลำดับจากมากไปน้อย\n3. กำหนด severity level (none/mild/severe)\n4. Focus แก้ไข features ที่มี severe drift ก่อน\n```\n\n### Time-based Distribution Analysis\n\nการติดตาม drift เมื่อเวลาผ่านไป:\n\n```\nPeriod 0 (Reference)\n    │\n    ▼\nPeriod 1 ──── PSI = 0.02 (none)\n    │\n    ▼\nPeriod 2 ──── PSI = 0.08 (none)\n    │\n    ▼\nPeriod 3 ──── PSI = 0.15 (mild) ⚠️\n    │\n    ▼\nPeriod 4 ──── PSI = 0.28 (severe) 🔴\n```\n\n---\n\n## LAB 3: Multivariate Drift Analysis\n\n### 🎯 วัตถุประสงค์\n- ตรวจจับ drift ที่เกิดจากความสัมพันธ์ระหว่าง features\n- ใช้ Dataset-level drift detection\n- วิเคราะห์ Correlation changes\n\n### ทฤษฎี: Multivariate Drift\n\n**ปัญหา**: Univariate methods อาจพลาด drift ที่เกิดจากการเปลี่ยนแปลงของความสัมพันธ์ระหว่าง features\n\n```\nตัวอย่าง:\nReference:                    Current:\n- Age mean = 35              - Age mean = 35      (เหมือนเดิม)\n- Income mean = 50000        - Income mean = 50000 (เหมือนเดิม)\n- Corr(Age, Income) = 0.8    - Corr(Age, Income) = 0.1 ← เปลี่ยน!\n\nUnivariate test: ไม่พบ drift\nMultivariate test: พบ drift ในความสัมพันธ์\n```\n\n### Methods สำหรับ Multivariate Drift\n\n#### 1. Correlation-based Analysis\n\n**ทฤษฎี**: เปรียบเทียบ correlation matrix ระหว่าง reference และ current data\n\n**Fisher's Z Transformation** สำหรับเปรียบเทียบ correlations:\n```\nZ = arctanh(r)\n\nZ-test statistic = (Z_ref - Z_cur) / SE\nwhere SE = sqrt(1/(n₁-3) + 1/(n₂-3))\n```\n\n**Code Reference**:\n```python\ndef correlation_drift_test(ref_df, cur_df, significance_level=0.05):\n    ref_corr = ref_df.corr()\n    cur_corr = cur_df.corr()\n    \n    for col1, col2 in feature_pairs:\n        r_ref = ref_corr.loc[col1, col2]\n        r_cur = cur_corr.loc[col1, col2]\n        \n        # Fisher's Z transformation\n        z_ref = np.arctanh(r_ref)\n        z_cur = np.arctanh(r_cur)\n        \n        # Z-test\n        se = np.sqrt(1/(n_ref-3) + 1/(n_cur-3))\n        z_stat = (z_ref - z_cur) / se\n        p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))\n```\n\n#### 2. PCA-based Analysis\n\n**ทฤษฎี**: ใช้ Principal Component Analysis เพื่อตรวจจับการเปลี่ยนแปลงของ multivariate structure\n\n```\nPCA Drift Detection:\n├── Explained Variance Ratio - variance ที่แต่ละ PC อธิบายได้\n├── Component Similarity - ทิศทางของ PC เหมือนเดิมหรือไม่\n└── Reconstruction Error - error เมื่อ reconstruct data\n```\n\n**Metrics ที่ใช้**:\n1. **Explained Variance Comparison**: เปรียบเทียบว่าแต่ละ PC อธิบาย variance เท่าเดิมหรือไม่\n2. **Component Similarity (Cosine Similarity)**: ดูว่า PC directions เหมือนเดิมหรือไม่\n3. **Reconstruction Error**: ใช้ reference PCA กับ current data แล้วดู error\n\n**Code Reference**:\n```python\ndef pca_drift_detection(ref_df, cur_df, n_components=None):\n    # Standardize\n    scaler = StandardScaler()\n    ref_scaled = scaler.fit_transform(ref_df)\n    cur_scaled = scaler.transform(cur_df)\n    \n    # Fit PCA on reference\n    pca_ref = PCA(n_components=n_components)\n    pca_ref.fit(ref_scaled)\n    \n    # Component similarities (using cosine similarity)\n    for i in range(n_components):\n        cos_sim = abs(np.dot(pca_ref.components_[i], pca_cur.components_[i]))\n        # ถ้า cos_sim < 0.9 = structure เปลี่ยน\n```\n\n#### 3. Mahalanobis Distance\n\n**ทฤษฎี**: วัดว่าข้อมูลใหม่อยู่ห่างจาก distribution ของ reference data เท่าไร โดยคำนึงถึง covariance\n\n```\nMahalanobis Distance = √((x - μ)ᵀ Σ⁻¹ (x - μ))\n\nโดยที่:\n- x = data point\n- μ = mean ของ reference\n- Σ = covariance matrix ของ reference\n```\n\n**การใช้งาน**:\n1. Fit covariance บน reference data\n2. คำนวณ Mahalanobis distance สำหรับทุก points\n3. เปรียบเทียบ distribution ของ distances\n\n**Code Reference**:\n```python\ndef mahalanobis_drift_detection(ref_df, cur_df, threshold_percentile=95):\n    # Fit covariance on reference\n    cov = EmpiricalCovariance().fit(ref_scaled)\n    \n    # Calculate distances\n    ref_distances = cov.mahalanobis(ref_scaled)\n    cur_distances = cov.mahalanobis(cur_scaled)\n    \n    # Compare distributions\n    ks_stat, ks_pval = stats.ks_2samp(ref_distances, cur_distances)\n    drift_detected = ks_pval < 0.05\n```\n\n### Comprehensive Multivariate Analysis\n\n```\nMultivariateriftDetector\n├── Correlation Analysis\n│   └── ตรวจจับ pairwise correlation changes\n├── PCA Analysis\n│   └── ตรวจจับ structure changes\n├── Mahalanobis Analysis\n│   └── ตรวจจับ distribution shift\n└── Consensus\n    └── รวมผลจากทุก methods\n```\n\n---\n\n## LAB 4: Drift Detection in Production Simulation\n\n### 🎯 วัตถุประสงค์\n- สร้าง simulated data stream ที่มี drift\n- ตรวจจับ sudden vs gradual drift\n- Implement sliding window monitoring\n\n### ทฤษฎี: Drift Patterns ใน Production\n\n#### Types of Drift\n\n```\n1. Sudden Drift (Abrupt)\n   ────────────┬────────────\n               │\n               ▼ Drift Point\n   เปลี่ยนทันที\n\n2. Gradual Drift\n   ────────────╱────────────\n   เปลี่ยนช้าๆ ตามเวลา\n\n3. Incremental Drift\n   ────┬───┬───┬───┬────────\n       │   │   │   │\n   เปลี่ยนเป็นขั้นบันได\n\n4. Seasonal/Recurring Drift\n   ────╲╱────╲╱────╲╱────\n   เปลี่ยนตาม pattern ซ้ำ\n```\n\n**Code Reference** - Data Stream Simulator:\n```python\nclass DataStreamSimulator:\n    def generate_stream(self, n_samples, drift_type='no_drift', drift_params=None):\n        if drift_type == 'sudden':\n            # เปลี่ยนทันทีที่ drift_point\n            data[:drift_point] = np.random.normal(base_mean, ...)\n            data[drift_point:] = np.random.normal(new_mean, ...)\n            \n        elif drift_type == 'gradual':\n            # Linear interpolation\n            for i in range(n_samples):\n                if drift_start <= i <= drift_end:\n                    progress = (i - drift_start) / (drift_end - drift_start)\n                    current_mean = base_mean + progress * (final_mean - base_mean)\n```\n\n### Sliding Window Drift Detection\n\n**หลักการ**:\n```\nTime ────────────────────────────────────────────▶\n\nReference Window          Test Window\n[═══════════════════]     [═════════]\n        ▲                      ▲\n        │                      │\n   Fixed/Slow moving    Current data\n```\n\n**Architecture**:\n```\nSlidingWindowDriftDetector\n├── reference_buffer (deque with maxlen)\n├── test_buffer (deque with maxlen)\n├── update(value)\n│   ├── Add to buffer\n│   ├── Check if ready\n│   └── Run drift detection\n├── Detection Methods\n│   ├── KS Test\n│   └── PSI\n└── Output\n    ├── drift_detected\n    ├── metrics\n    └── history\n```\n\n**Code Reference**:\n```python\nclass SlidingWindowDriftDetector:\n    def __init__(self, reference_window_size=200, test_window_size=100):\n        self.reference_buffer = deque(maxlen=reference_window_size)\n        self.test_buffer = deque(maxlen=test_window_size)\n    \n    def update(self, value, timestamp=None):\n        # เพิ่มค่าใน buffers\n        if not self.is_initialized:\n            self.reference_buffer.append(value)\n            return {'status': 'initializing'}\n        \n        self.test_buffer.append(value)\n        \n        if len(self.test_buffer) < self.test_window_size:\n            return {'status': 'collecting'}\n        \n        # ทำ drift detection\n        ref_array = np.array(self.reference_buffer)\n        test_array = np.array(self.test_buffer)\n        \n        psi = self.calculate_psi(ref_array, test_array)\n        drift_detected = psi > self.psi_threshold\n        \n        return {\n            'drift_detected': drift_detected,\n            'psi': psi\n        }\n```\n\n### Adaptive Reference Window\n\n**ปัญหาของ Fixed Reference**:\n- Gradual drift อาจตรวจไม่เจอ เพราะ reference เก่าเกินไป\n- ต้อง adapt reference เมื่อ detect drift\n\n**วิธีแก้**:\n```python\nclass AdaptiveDriftDetector:\n    def adapt_reference(self):\n        \"\"\"ปรับ reference window เมื่อ confirm drift\"\"\"\n        # ผสม old reference กับ new data\n        old_weight = 0.5\n        \n        # เพิ่มบางส่วนจาก old reference\n        for val in old_ref[-n_old:]:\n            self.reference_buffer.append(val)\n        \n        # เพิ่ม new data\n        for val in new_data:\n            self.reference_buffer.append(val)\n```\n\n### Page-Hinkley Test\n\n**ทฤษฎี**: Algorithm สำหรับ detect mean shift ใน streaming data\n\n```\nAlgorithm:\n1. Update running mean: μ_t = α·μ_{t-1} + (1-α)·x_t\n2. Update cumulative sum: S_t = S_{t-1} + (x_t - μ_t - δ)\n3. Track min/max of S_t\n4. Detect if S_t - min(S) > λ (upward shift)\n         หรือ max(S) - S_t > λ (downward shift)\n```\n\n**Code Reference**:\n```python\nclass PageHinkleyDetector:\n    def update(self, value):\n        # Update mean (exponential moving average)\n        self.mean = self.alpha * self.mean + (1 - self.alpha) * value\n        \n        # Update cumulative sum\n        self.sum += value - self.mean - self.delta\n        \n        # Update min/max\n        self.min_sum = min(self.min_sum, self.sum)\n        self.max_sum = max(self.max_sum, self.sum)\n        \n        # Detection\n        ph_positive = self.sum - self.min_sum  # Upward shift\n        ph_negative = self.max_sum - self.sum  # Downward shift\n        \n        drift_detected = (ph_positive > self.lambda_) or (ph_negative > self.lambda_)\n```\n\n### Comparison of Methods\n\n| Method | Pros | Cons | Best For |\n|--------|------|------|----------|\n| Sliding Window | Simple, intuitive | Fixed reference | Sudden drift |\n| Adaptive | Handles gradual drift | More complex | Production |\n| Page-Hinkley | Low memory, fast | Mean shift only | Real-time |\n\n---\n\n## LAB 5: Custom Metrics & Drift Thresholds\n\n### 🎯 วัตถุประสงค์\n- สร้าง custom drift metrics\n- ปรับ threshold ตาม business requirements\n- Handle false positives/negatives\n\n### ทฤษฎี: Threshold Selection\n\n**ปัญหา**: Default thresholds อาจไม่เหมาะกับทุก use case\n\n```\nTrade-off:\n                    Threshold\n           Low ◄─────────────────▶ High\n           \nSensitivity: High                    Low\nFalse Positive: High                 Low\nFalse Negative: Low                  High\n```\n\n### Custom Drift Metrics\n\n#### 1. Combined Score\n```python\ndef combined_score(reference, current, weights=None):\n    \"\"\"รวมหลาย metrics เข้าด้วยกัน\"\"\"\n    if weights is None:\n        weights = {\n            'psi': 0.3,\n            'wasserstein': 0.3,\n            'mean_shift': 0.2,\n            'percentile': 0.2\n        }\n    \n    # Normalize แต่ละ metric ให้อยู่ในช่วง 0-1\n    psi = min(calculate_psi(reference, current), 1.0)\n    wasserstein = min(normalized_wasserstein(reference, current) / 3, 1.0)\n    mean_shift = min(mean_shift_ratio(reference, current) / 3, 1.0)\n    percentile = min(percentile_shift(reference, current) / 3, 1.0)\n    \n    score = (weights['psi'] * psi + \n             weights['wasserstein'] * wasserstein +\n             weights['mean_shift'] * mean_shift +\n             weights['percentile'] * percentile)\n    \n    return score\n```\n\n#### 2. Jensen-Shannon Divergence\n```python\ndef jensen_shannon_divergence(reference, current, bins=10):\n    \"\"\"Symmetric version of KL Divergence\"\"\"\n    # สร้าง normalized histograms\n    ref_hist = np.histogram(reference, bins=bins, density=True)[0]\n    cur_hist = np.histogram(current, bins=bins, density=True)[0]\n    \n    # Average distribution\n    m = 0.5 * (ref_hist + cur_hist)\n    \n    # JS Divergence = 0.5 * (KL(P||M) + KL(Q||M))\n    js = 0.5 * (kl_divergence(ref_hist, m) + kl_divergence(cur_hist, m))\n    return js\n```\n\n### Threshold Optimization\n\n#### F1-based Optimization\n\n```python\ndef find_optimal_threshold(scenarios, metric_func, thresholds, optimize_for='f1'):\n    \"\"\"หา threshold ที่ให้ F1 score สูงสุด\"\"\"\n    results = []\n    \n    for t in thresholds:\n        y_true = [s['has_drift'] for s in scenarios]\n        y_pred = [metric_func(s['reference'], s['current']) > t for s in scenarios]\n        \n        precision = precision_score(y_true, y_pred)\n        recall = recall_score(y_true, y_pred)\n        f1 = f1_score(y_true, y_pred)\n        \n        results.append({'threshold': t, 'f1': f1})\n    \n    # หา optimal\n    optimal = max(results, key=lambda x: x['f1'])\n    return optimal\n```\n\n#### Cost-based Optimization\n\n**ทฤษฎี**: ปรับ threshold ตาม cost ของ false positive vs false negative\n\n```\nTotal Cost = FP_count × FP_cost + FN_count × FN_cost\n\nScenarios:\n├── High FN cost (เช่น fraud detection)\n│   └── ใช้ Low threshold → detect more, accept false alarms\n├── High FP cost (เช่น expensive retraining)\n│   └── ใช้ High threshold → conservative detection\n└── Balanced cost\n    └── Optimize for F1\n```\n\n**Code Reference**:\n```python\nclass BusinessDriftThreshold:\n    def __init__(self, false_positive_cost=1, false_negative_cost=10):\n        self.fp_cost = false_positive_cost\n        self.fn_cost = false_negative_cost\n    \n    def calculate_total_cost(self, y_true, y_pred):\n        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()\n        return fp * self.fp_cost + fn * self.fn_cost\n    \n    def find_cost_optimal_threshold(self, scenarios, metric_func, thresholds):\n        costs = []\n        for t in thresholds:\n            y_pred = [metric_func(s['reference'], s['current']) > t \n                     for s in scenarios]\n            cost = self.calculate_total_cost(y_true, y_pred)\n            costs.append({'threshold': t, 'cost': cost})\n        \n        optimal = min(costs, key=lambda x: x['cost'])\n        return optimal\n```\n\n### Handling False Positives/Negatives\n\n#### Ensemble Approach\n```python\nclass RobustDriftDetector:\n    \"\"\"ใช้หลาย methods ร่วมกัน\"\"\"\n    \n    def detect(self, reference, current):\n        all_metrics = self._calculate_all_metrics(reference, current)\n        \n        # Majority voting\n        drift_votes = sum(\n            1 for m, v in all_metrics.items() \n            if v > self.thresholds[m]\n        )\n        \n        # ต้อง 3/4 methods agree\n        ensemble_drift = drift_votes >= 3\n        \n        return ensemble_drift\n```\n\n#### Confirmation Mechanism\n```python\ndef detect_with_confirmation(self, reference, current):\n    \"\"\"ต้อง detect หลายครั้งติดกันถึงยืนยัน\"\"\"\n    \n    potential_drift = self.primary_metric(reference, current) > threshold\n    \n    if potential_drift:\n        self.consecutive_count += 1\n    else:\n        self.consecutive_count = 0\n    \n    # Confirmed ถ้า detect 3 ครั้งติดกัน\n    confirmed = self.consecutive_count >= 3\n    return confirmed\n```\n\n### Best Practices สำหรับ Threshold Setting\n\n```\n1️⃣ DOMAIN-SPECIFIC THRESHOLDS\n   - ไม่ใช้ default โดยไม่ validate\n   - ทดสอบกับ labeled data\n   - ปรึกษา domain experts\n\n2️⃣ COST-BASED OPTIMIZATION\n   - พิจารณา cost ของ FP vs FN\n   - FN แพง → Lower threshold\n   - FP แพง → Higher threshold\n\n3️⃣ ENSEMBLE APPROACH\n   - ใช้หลาย metrics ร่วมกัน\n   - Voting mechanism ลด false positives\n\n4️⃣ CONFIRMATION MECHANISM\n   - Require consecutive detections\n   - ป้องกัน temporary spikes\n\n5️⃣ PERIODIC REVIEW\n   - Review thresholds เป็นระยะ\n   - Business requirements อาจเปลี่ยน\n```\n\n---\n\n## LAB 6: End-to-End Monitoring Pipeline\n\n### 🎯 วัตถุประสงค์\n- รวมทุก components เข้าด้วยกัน\n- สร้าง automated monitoring workflow\n- Integrate กับ MLflow\n\n### Architecture Overview\n\n```\n┌─────────────────────────────────────────────────────────────┐\n│                 Drift Monitoring Pipeline                    │\n├─────────────────────────────────────────────────────────────┤\n│                                                              │\n│  ┌────────────────┐    ┌────────────────┐                   │\n│  │  Data Source   │───▶│  Data Buffer   │                   │\n│  └────────────────┘    └───────┬────────┘                   │\n│                                │                             │\n│                    ┌───────────▼───────────┐                │\n│                    │   Drift Calculator    │                │\n│                    └───────────┬───────────┘                │\n│                                │                             │\n│           ┌────────────────────┼────────────────────┐       │\n│           │                    │                    │       │\n│           ▼                    ▼                    ▼       │\n│  ┌─────────────┐      ┌─────────────┐      ┌─────────────┐ │\n│  │   Alert     │      │   Report    │      │   MLflow    │ │\n│  │   Manager   │      │   Generator │      │   Tracker   │ │\n│  └─────────────┘      └─────────────┘      └─────────────┘ │\n│                                                              │\n└─────────────────────────────────────────────────────────────┘\n```\n\n### Core Components\n\n#### 1. Data Classes\n\n```python\nfrom dataclasses import dataclass\nfrom enum import Enum\n\nclass AlertSeverity(Enum):\n    INFO = \"info\"\n    WARNING = \"warning\"\n    CRITICAL = \"critical\"\n\nclass DriftType(Enum):\n    NONE = \"none\"\n    MILD = \"mild\"\n    MODERATE = \"moderate\"\n    SEVERE = \"severe\"\n\n@dataclass\nclass DriftResult:\n    timestamp: datetime\n    feature: str\n    drift_detected: bool\n    drift_type: DriftType\n    psi: float\n    ks_statistic: float\n    ks_pvalue: float\n    reference_mean: float\n    current_mean: float\n\n@dataclass\nclass Alert:\n    timestamp: datetime\n    severity: AlertSeverity\n    message: str\n    details: Dict\n    acknowledged: bool = False\n```\n\n#### 2. Configuration\n\n```python\n@dataclass\nclass MonitoringConfig:\n    reference_window_size: int = 1000\n    current_window_size: int = 200\n    psi_mild_threshold: float = 0.1\n    psi_moderate_threshold: float = 0.2\n    psi_severe_threshold: float = 0.25\n    ks_significance: float = 0.05\n    check_interval_seconds: int = 60\n    alert_cooldown_minutes: int = 30\n    features_to_monitor: List[str] = field(default_factory=list)\n```\n\n#### 3. Data Buffer\n\n```python\nclass DataBuffer:\n    \"\"\"Buffer สำหรับเก็บ reference และ current data\"\"\"\n    \n    def __init__(self, config: MonitoringConfig):\n        self.reference_data: Dict[str, deque] = {}\n        self.current_data: Dict[str, deque] = {}\n    \n    def initialize(self, reference_df: pd.DataFrame):\n        \"\"\"Initialize with reference data\"\"\"\n        for feature in self.config.features_to_monitor:\n            self.reference_data[feature] = deque(\n                reference_df[feature].values,\n                maxlen=self.config.reference_window_size\n            )\n    \n    def add_data(self, data: Dict[str, float]):\n        \"\"\"Add new data point\"\"\"\n        for feature, value in data.items():\n            self.current_data[feature].append(value)\n```\n\n#### 4. Alert Manager\n\n```python\nclass AlertManager:\n    \"\"\"จัดการ alerts พร้อม cooldown\"\"\"\n    \n    def __init__(self, config: MonitoringConfig):\n        self.alerts: List[Alert] = []\n        self.last_alert_time: Dict[str, datetime] = {}\n    \n    def should_alert(self, feature: str) -> bool:\n        \"\"\"Check cooldown\"\"\"\n        if feature not in self.last_alert_time:\n            return True\n        elapsed = datetime.now() - self.last_alert_time[feature]\n        return elapsed > timedelta(minutes=self.config.alert_cooldown_minutes)\n    \n    def create_alert(self, drift_result: DriftResult) -> Optional[Alert]:\n        if not drift_result.drift_detected:\n            return None\n        \n        if not self.should_alert(drift_result.feature):\n            return None\n        \n        # Determine severity\n        severity = self._determine_severity(drift_result.drift_type)\n        \n        alert = Alert(\n            timestamp=drift_result.timestamp,\n            severity=severity,\n            message=f\"Drift detected in {drift_result.feature}\",\n            details=drift_result.to_dict()\n        )\n        \n        self.alerts.append(alert)\n        self.last_alert_time[drift_result.feature] = datetime.now()\n        \n        return alert\n```\n\n#### 5. Main Pipeline\n\n```python\nclass DriftMonitoringPipeline:\n    def __init__(self, config: MonitoringConfig):\n        self.config = config\n        self.data_buffer = DataBuffer(config)\n        self.alert_manager = AlertManager(config)\n        self.drift_calculator = DriftCalculator()\n        self.results_history: List[DriftResult] = []\n    \n    def initialize(self, reference_data: pd.DataFrame):\n        self.data_buffer.initialize(reference_data)\n    \n    def process_batch(self, batch_data: pd.DataFrame) -> List[DriftResult]:\n        results = []\n        \n        # Add data to buffer\n        for idx, row in batch_data.iterrows():\n            self.data_buffer.add_data(row.to_dict())\n        \n        # Check if ready\n        if not self.data_buffer.is_current_ready():\n            return results\n        \n        # Detect drift for each feature\n        for feature in self.config.features_to_monitor:\n            result = self._detect_drift_for_feature(feature)\n            if result:\n                results.append(result)\n                \n                # Create alert if needed\n                if result.drift_detected:\n                    self.alert_manager.create_alert(result)\n        \n        return results\n```\n\n### Report Generation\n\n```python\nclass ReportGenerator:\n    def generate_html_report(self, output_path: str):\n        \"\"\"Generate HTML report\"\"\"\n        summary = self.pipeline.get_summary_report()\n        \n        html_content = f\"\"\"\n        <html>\n        <head><title>Drift Monitoring Report</title></head>\n        <body>\n            <h1>Drift Monitoring Report</h1>\n            <p>Generated: {datetime.now()}</p>\n            \n            <h2>Summary</h2>\n            <p>Total Checks: {summary['total_checks']}</p>\n            <p>Drifts Detected: {summary['total_drifts_detected']}</p>\n            \n            <h2>Feature Summary</h2>\n            <table>\n                <tr><th>Feature</th><th>PSI</th><th>Status</th></tr>\n                {self._generate_feature_rows(summary)}\n            </table>\n            \n            <h2>Active Alerts</h2>\n            {self._generate_alerts_section()}\n        </body>\n        </html>\n        \"\"\"\n        \n        with open(output_path, 'w') as f:\n            f.write(html_content)\n```\n\n### MLflow Integration\n\n```python\nclass MLflowDriftTracker:\n    def __init__(self, experiment_name: str = \"drift_monitoring\"):\n        mlflow.set_experiment(experiment_name)\n    \n    def log_drift_result(self, result: DriftResult):\n        mlflow.log_metric(f\"{result.feature}_psi\", result.psi)\n        mlflow.log_metric(f\"{result.feature}_drift\", 1 if result.drift_detected else 0)\n        mlflow.log_param(f\"{result.feature}_drift_type\", result.drift_type.value)\n    \n    def log_summary(self, summary: Dict):\n        mlflow.log_metric(\"total_drifts\", summary.get('total_drifts_detected', 0))\n        for feature, data in summary.get('feature_summary', {}).items():\n            mlflow.log_metric(f\"{feature}_drift_rate\", data['drift_rate'])\n    \n    def log_artifact(self, artifact_path: str):\n        mlflow.log_artifact(artifact_path)\n```\n\n---\n\n## สรุปและ Best Practices\n\n### Summary Table\n\n| Lab | หัวข้อ | Key Concepts |\n|-----|--------|--------------|\n| 1 | Understanding Data Drift | Covariate/Concept Shift, KS/PSI/Wasserstein |\n| 2 | Feature Drift Detection | Per-feature analysis, Numerical vs Categorical |\n| 3 | Multivariate Drift | Correlation, PCA, Mahalanobis Distance |\n| 4 | Production Simulation | Streaming, Sliding Window, Page-Hinkley |\n| 5 | Custom Metrics & Thresholds | Optimization, Cost-based, Ensemble |\n| 6 | End-to-End Pipeline | Architecture, Alerting, Reporting, MLflow |\n\n### Decision Framework\n\n```\nเมื่อพบ Drift ควรทำอย่างไร?\n\n1. Assess Severity\n   ├── MILD → Monitor closely\n   ├── MODERATE → Investigate root cause\n   └── SEVERE → Immediate action required\n\n2. Investigate Root Cause\n   ├── Data collection issue?\n   ├── Upstream data change?\n   ├── Real-world change?\n   └── Seasonal pattern?\n\n3. Decide Action\n   ├── Retrain model\n   ├── Update feature engineering\n   ├── Adjust thresholds\n   └── Business process change\n```\n\n### Production Checklist\n\n```\n□ Define monitoring strategy\n  □ Which features to monitor?\n  □ What thresholds to use?\n  □ How often to check?\n\n□ Implement detection\n  □ Choose appropriate methods\n  □ Handle both numerical and categorical\n  □ Consider multivariate drift\n\n□ Set up alerting\n  □ Define severity levels\n  □ Configure notification channels\n  □ Set cooldown periods\n\n□ Create reporting\n  □ Automated dashboards\n  □ Periodic reports\n  □ Historical analysis\n\n□ Plan remediation\n  □ When to retrain?\n  □ How to validate new model?\n  □ Rollback procedures\n```\n\n### Final Thoughts\n\nการทำ Drift Detection ที่ดีต้อง:\n\n1. **เข้าใจ Business Context**: Drift ที่สำคัญใน domain หนึ่งอาจไม่สำคัญใน domain อื่น\n\n2. **ใช้หลาย Methods**: ไม่มี method ใดที่สมบูรณ์แบบ ควรใช้หลาย methods ร่วมกัน\n\n3. **Tune Thresholds**: Default thresholds มักไม่เหมาะ ต้องปรับตาม use case\n\n4. **Automate**: การทำ manual monitoring ไม่ยั่งยืน ต้องมี automated pipeline\n\n5. **Monitor the Monitor**: ตรวจสอบว่า monitoring system ทำงานถูกต้อง\n\n---\n\n## แหล่งเรียนรู้เพิ่มเติม\n\n- [Evidently AI Documentation](https://docs.evidentlyai.com/)\n- [NannyML Documentation](https://docs.nannyml.com/)\n- [Alibi Detect](https://docs.seldon.io/projects/alibi-detect/)\n- [Great Expectations](https://docs.greatexpectations.io/)\n\n---\n\n*เอกสารนี้จัดทำเพื่อประกอบการเรียนการสอนวิชา MLOps*"
    }
  ],
 
}


In [6]:
# Simple extraction
text = response["content"][0]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

# 📚 Data Drift Detection: Complete Guide for MLOps

## สารบัญ
1. [บทนำ: ทำความเข้าใจ Data Drift](#บทนำ-ทำความเข้าใจ-data-drift)
2. [LAB 1: Understanding Data Drift Concepts](#lab-1-understanding-data-drift-concepts)
3. [LAB 2: Feature Drift Detection](#lab-2-feature-drift-detection)
4. [LAB 3: Multivariate Drift Analysis](#lab-3-multivariate-drift-analysis)
5. [LAB 4: Drift Detection in Production Simulation](#lab-4-drift-detection-in-production-simulation)
6. [LAB 5: Custom Metrics & Drift Thresholds](#lab-5-custom-metrics--drift-thresholds)
7. [LAB 6: End-to-End Monitoring Pipeline](#lab-6-end-to-end-monitoring-pipeline)
8. [สรุปและ Best Practices](#สรุปและ-best-practices)

---

## บทนำ: ทำความเข้าใจ Data Drift

### Data Drift คืออะไร?

**Data Drift** (หรือ Dataset Shift) คือปรากฏการณ์ที่ข้อมูลในระบบ production มีการเปลี่ยนแปลงไปจากข้อมูลที่ใช้ train โมเดล ซึ่งเป็นหนึ่งในสาเหตุหลักที่ทำให้ ML Model มีประสิทธิภาพลดลงเมื่อเวลาผ่านไป

```
Training Time                    Production Time